# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [6]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.
The dataset I'm using for this project is the Heart Failure Prediction Dataset from kaggle.

fedesoriano. (September 2021). Heart Failure Prediction Dataset. Retrieved [2021-10-18] from https://www.kaggle.com/fedesoriano/heart-failure-prediction.

The task with this dataset is to predict whether a person will develop a heart disease with a set of 11 diagnostic features. This dataset is a combination of five independent heart disease datasets containing 918 observations of patients. The target column "Heart Disease" is nearly balanced in this dataset with 510 patients with and 408 patients without cardiovascular diseases.

The features in this dataset are:

General information
- Age of the patient in years; with the youngest patient being 28 and the oldest 77 years old
- Sex of the patient; the majority of the patients (79%) being male

blood tests
- Cholesterol in serum [mm/dl]; an indicator for ateriosclerosis
- fasting blood sugar level; a boolean value if the blood sugar is elevated (>120mg/dl) or not, which is an indicator for diabetes

medical history
- type of chest pain the patient is experiencing in four categories (TA: typical angina, ATA: atypical angina, NAP: non-anginal pain, ASY: asymptomatic)

ECG and cardiac stress testing
- resting blood pressure [mm Hg]
- resting ECG results in three categories (Normal, ST: ST-T wave abnormality, LVH : left ventricular hypertrophy)
- oldpeak: the depression between S and T peak
- maximum heartrate under cardiac stress
- exercise-induced angina
- ST-slope of te peak exercise in three categories (Up, flat and down)

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

### setup workspace and experiment

Use Workspace.from_config() to get the workspace configuration in the VM.
Set up an experiment with the name "heart-failure-experiment".

In [8]:
ws = Workspace.from_config()

# print some information about the workspace
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# choose a name for experiment
experiment_name = 'heart-failure-experiment'

experiment=Experiment(ws, experiment_name)

Workspace name: test-workspace
Azure region: eastus2
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-161432


### create a cluster

check, whether a compute cluster already exists. If not, create one.

In [9]:
cluster_name = "expcluster"

# Use existing cluster, if it exists
try:
    compute_target = ComputeTarget(workspace=ws, name = cluster_name)
    print('Found existing cluster, use it!')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',
                                                          max_nodes=6, min_nodes=1)
    compute_target = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)
compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it!
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### create Dataset

Since I need the dataset for the AutoMl run as well as for the HyperDriveRun I configured the upload of local data and registering of the dataset in the function read_data() from the train.py script.

The preprocessing of the data in regard of cleaning and encoding categorical data is defined in the function prepare_data() in the train.py script.

In [1]:
from train import read_data
dataset=read_data()

Uploading an estimated of 4 files
Uploading ./data/.amlignore
Uploaded ./data/.amlignore, 1 files out of an estimated total of 4
Uploading ./data/.amlignore.amltmp
Uploaded ./data/.amlignore.amltmp, 2 files out of an estimated total of 4
Uploading ./data/heart.csv
Uploaded ./data/heart.csv, 3 files out of an estimated total of 4
Uploading ./data/heart_failure_clinical_records_dataset.csv
Uploaded ./data/heart_failure_clinical_records_dataset.csv, 4 files out of an estimated total of 4
Uploaded 4 files
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/9ff33213-41e6-427b-b9a7-7ef59f7924a0/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [2]:
df = dataset.to_pandas_dataframe()
df.describe()

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.0,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,0.789760,132.396514,198.799564,0.233115,136.809368,0.0,0.887364,0.361656,0.553377,0.540305,0.188453,0.221133,0.050109,0.204793,0.601307,0.193900
std,9.432617,0.407701,18.514154,109.384145,0.423046,25.460334,0.0,1.066570,0.607056,0.497414,0.498645,0.391287,0.415236,0.218289,0.403770,0.489896,0.395567
min,28.000000,0.000000,0.000000,0.000000,0.000000,60.000000,0.0,-2.600000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.000000,1.000000,120.000000,173.250000,0.000000,120.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,54.000000,1.000000,130.000000,223.000000,0.000000,138.000000,0.0,0.600000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,60.000000,1.000000,140.000000,267.000000,0.000000,156.000000,0.0,1.500000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,77.000000,1.000000,200.000000,603.000000,1.000000,202.000000,0.0,6.200000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

For the automl_settings I used a timeout of 20minutes, to ensure completion of the experiment run before the VM times out.
The maximum concurrent_iterations are 5, meaning that five iterations can run in parallel. This is less than the maximum number of nodes of my compute_cluster. The primary metric to evaluate the best model is precision, which I chose, since the training_data is imbalanced. Instead of splitting the dataset into training and test data, I use a 5-fold crossvalidation. The early stooping logic is enabled, which stops an iteration if there is no improvement in the primary metric after 31 iterations.

I configured the AutoMl run, that it runs on the above specified compute cluster ("compute_target").
The AutoML experiment will run a classification task on the heart-failure-dataset with the target column "HeartDisease".
 I turned the featurization off, since the data is already preprocessed. 

In [16]:
from azureml.train.automl import AutoMLConfig
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric": 'accuracy',
    "n_cross_validations": 5,
    "enable_early_stopping": True,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task="classification",
                             training_data=dataset,
                             label_column_name="HeartDisease",
                             path = './automl_run',
                             featurization = "off",
                             **automl_settings
                            )

In [17]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-experiment,AutoML_ac4bfb14-36a7-40e2-8dac-11314a8a2737,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [18]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_automl_run = remote_run.get_best_child()
print("best run details: ", best_automl_run.get_details())
print("best run metrics: ", best_automl_run.get_metrics())

In [ ]:
#TODO: Save the best model
for f in best_automl_run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', 'automl_model.pkl')
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_automl_run.download_file(name=f, output_file_path=output_file_path)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model_name = best_automl_run.properties['model_name']
description = "AutoML heart-failure classification model"

model = remote_run.register_model(model_name = model_name,
                                  description = description)

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Model

service_name = 'heart-failure-automl-service'

inference_config = InferenceConfig(entry_script="score.py")
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,
                                                enable_app_insights=True)

service = Model.deplay(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# take any line from the dataset and run it against the webservice

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
